<a href="https://colab.research.google.com/github/atarasaki/DSWP-editados/blob/main/Notebooks/2020-10-21-fifa-mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b><i>mini task - FIFA dataset 20/10/2020</i></b></h1></center>

Nelio Machado, [20.10.20 17:46]

*Mini-task*: apliquem o IsolationForest (primeiro método para detecção de outliers) para o dataframe FIFA_algumas_features (variáveis numéricas). Qual a conclusão?

Qual sua conclusão em termos de outliers?

Como ficou sua "outlier region"?


___
# **Carregar a library Pandas e verificar a versão**

In [ ]:
# Carrega a library Pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

print(f'Versão do Pandas: {pd.__version__}')
print(f'Versão do NumPy.: {np.__version__}')

# Configurações
> Podemos configurar o pandas de forma a tornar nosso trabalho mais produtivo. Podemos configurar, por exemplo, o número de LINHAS e COLUNAS a ser mostrado, precisão dos números float. Vamos ver com mais detalhes a seguir.

Fonte: [5 Advanced Features of Pandas and How to Use Them](https://www.kdnuggets.com/2019/10/5-advanced-features-pandas.html).

In [ ]:
d_configuracao = {
    'display.max_columns': 1000,
    'display.expand_frame_repr': True,
    'display.max_rows': 10,
    'display.precision': 2,
    'display.show_dimensions': True
                  }

for op, value in d_configuracao.items():
    pd.set_option(op, value)
    print(op, value)

___
# **Exercícios**

## Exercício 9 - modificado : dataset FIFA_algumas_features

*Mini-task*: apliquem o IsolationForest (primeiro método para detecção de outliers) para o dataframe FIFA_algumas_features (variáveis numéricas). Qual a conclusão?

Qual sua conclusão em termos de outliers?

Como ficou sua "outlier region"?



* -------------------------------------------------------- 09/10/2020

https://www.kaggle.com/thec03u5/fifa-18-demo-player-dataset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

print(f'Versão do Pandas: {pd.__version__}')
print(f'Versão do NumPy.: {np.__version__}')

In [ ]:
d_configuracao = {
    'display.max_columns': 1000,
    'display.expand_frame_repr': True,
    'display.max_rows': 10,
    'display.precision': 2,
    'display.show_dimensions': True,
    'display.float_format': (lambda x: '%.2f' % x)
                  }

for op, value in d_configuracao.items():
    pd.set_option(op, value)
    print(op, value)

## 1) Carregue o arquivo FIFA.csv (está na área de Dataframes do curso);

In [ ]:
df_fifa = pd.read_csv('https://raw.githubusercontent.com/atarasaki/DSWP/master/Dataframes/FIFA.csv')

alternativa

df_fifa = pd.read_csv('https://raw.githubusercontent.com/atarasaki/DSWP/master/Dataframes/FIFA.csv', index_col ='ID')

df_fifa.set_index('ID', inplace=True)

## 2) Que colunas podem previamente ser eliminadas da análise? Porque identificar o que pode ser eliminado é importante?

In [ ]:
df_fifa.head()

In [ ]:
df_fifa.columns

De imediato, colunas 'Photo', 'Flag', 'Club Logo', 'Real Face','Jersey Number' podem ser eliminados
Por falta de significado, também podem ser excluídas as colunas 'LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM','LM','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB','LB','LCB','CB','RCB','RB'

In [ ]:
df_fifa_org = df_fifa.copy

In [ ]:
df_fifa.drop(inplace=True, columns=['Photo','Flag','Club Logo','Real Face','Jersey Number','LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM','LM','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB','LB','LCB','CB','RCB','RB'])
df_fifa.head()

## 3) Qual o dtype de cada variável/atributo do dataframe?

In [ ]:
df_fifa.info()

In [ ]:
df_fifa.select_dtypes(include=['number']).columns

In [ ]:
df_fifa.select_dtypes(include=['object']).columns

In [ ]:
df_fifa.describe(include='all')

## 4) Se alguma variávável/atributo é do tipo string (object) e supostamente deveria ser numérica, como alteramos o tipo?

df_fifa['nome variável'].astype( 'int64' )

In [ ]:
# expressões regulares : 'new_wage' recebe parte numérica de 'Wage'
df_fifa['wage_val'] = df_fifa['Wage'].str.extract( '([\d]+)' ).astype('float64')
df_fifa.head()

In [ ]:
# função para associar potência de dez aos caracteres das colunas 'Value', 'Wage', 'Release Clause'
func_multiplicador = lambda x: 1000.0 if x == 'K' else ( 1000000.0 if x == 'M' else 1.0 )

In [ ]:
df_fifa['wage_sufix'] = df_fifa['Wage'].str.extract( '[\d]+(\w)' ).astype('str')
df_fifa.head()

In [ ]:
df_fifa['new_wage'] = df_fifa['wage_val'] * df_fifa['wage_sufix'].map( func_multiplicador )
df_fifa.drop(inplace=True, columns=['wage_val','wage_sufix'])
df_fifa.head()

conversão análoga para 'Value' e 'Release Clause'

In [ ]:
# campo 'Value'
df_fifa['value_value'] = df_fifa['Value'].str.extract( '([\d]+[\.]*[\d]+)' ).astype('float64')
df_fifa['value_sufix'] = df_fifa['Value'].str.extract( '[\d]+[\.]*[\d]+(\w)' ).astype('str')
df_fifa['new_value'] = df_fifa['value_value'] * df_fifa['value_sufix'].map( func_multiplicador ).head()
df_fifa.drop(inplace=True, columns=['value_value','value_sufix'])

df_fifa.head()

In [ ]:
# campo 'Release Clause'
df_fifa['release_value'] = df_fifa['Release Clause'].str.extract( '([\d]+[\.]*[\d]+)' ).astype('float64')
df_fifa['release_sufix'] = df_fifa['Release Clause'].str.extract( '[\d]+[\.]*[\d]+(\w)' ).astype('str')
df_fifa['new_release_value'] = df_fifa['release_value'] * df_fifa['release_sufix'].map( func_multiplicador ).head()
df_fifa.drop(inplace=True, columns=['release_value','release_sufix'])

df_fifa.head()

In [ ]:
# elimina as colunas originais
df_fifa.drop(inplace=True, columns=['Wage','Value','Release Clause'])
df_fifa.head()

## 5) Normalize os nomes das colunas, ou seja, renomeie o nome das colunas para minúsculo;

In [ ]:
# passo 1 : extração do nome das colunas
s_colunas = df_fifa.columns
s_colunas

In [ ]:
# passo 2 : transformar o nome das colunas em minúsculas
s_colunas.str.lower()

In [ ]:
# passo 3 : aplicar o nome das colunas em minúsculas
df_fifa.columns = s_colunas.str.lower()
df_fifa.head()

## 6) Há Missing values nos dados? Se sim, o qual sua proposta (proposta do grupo) para tratar estes Missing values?

In [ ]:
# gera nome para a coluna 0
df_fifa.rename( columns={'unnamed: 0':'seq'}, inplace=True )
df_fifa.head()

In [ ]:
# mostra os valores das colunas
d_col_val = {}
for col in df_fifa.columns:
  d_col_val[col] = df_fifa[col].unique()

d_col_val

In [ ]:
# list comprehension para mostrar as colunas com missing values e as quantidades correspondentes
l_missing_values_col = [ col for col in df_fifa.columns if df_fifa[col].isnull().sum() > 0 ]
l_missing_values_col

In [ ]:
df_fifa.dtypes.unique()

* Para as variáveis numéricas, optar pela mediana, de modo a não alterar distribuição.

* Para as variáveis categóricas, analisar cada uma e determinar conteúdo

In [ ]:
# variáveis numéricas

In [ ]:
# teste
df_fifa['sprintspeed'].median()

In [ ]:
# aplicação da mediana para variáveis numéricas
for col in df_fifa.columns:
  if ( df_fifa[col].dtype == 'int64' ) or ( df_fifa[col].dtype == 'float64' ):
    df_fifa[col].fillna( df_fifa[col].median(), inplace=True )

df_fifa.isnull().sum()

In [ ]:
# avaliação da representatividade dos missing values
ds_lin, ds_col = df_fifa.shape
for col in df_fifa.columns:
  soma_mv = df_fifa[col].isnull().sum()
  if soma_mv > 0:
    print(col, ': tipo ', df_fifa[col].dtype, ': qt ', soma_mv, ' : ', (soma_mv/ds_lin)*100, ' %' )

In [ ]:
# variáveis categóricas

In [ ]:
# transformação de variáveis numéricas formatadas - height
df_fifa['height'].unique()

In [ ]:
s1 = "6'8"
s1.split("'",1)[1]

In [ ]:
# conversão de altura : feet'inch -> metros
def func_converte_altura( altura_ingles ):
  '''
  conversão de altura : feet'inch -> metros
  '''
  altura_aux = altura_ingles.split( "'", 1 )
  altura_feet = altura_aux[0]
  try:
    altura_inch = altura_aux[1]
  except:
    altura_inch = 0.0
  return ( ( float( altura_feet ) * 12 ) + float( altura_inch ) ) * 0.0254


In [ ]:
df_fifa['new_height'] = df_fifa['height'].astype('str').map( func_converte_altura )
df_fifa.head()


In [ ]:
df_fifa['new_height'].dtype

In [ ]:
df_fifa['new_height'].fillna( df_fifa['new_height'].median(), inplace=True )
df_fifa.head()

In [ ]:
df_fifa['new_height'].isnull().sum()

In [ ]:
# transformação de variáveis numéricas formatadas - weight
df_fifa['weight'].unique()

In [ ]:
# conversão de peso : lbs -> kg
def func_converte_peso( peso_lbs ):
  '''
  conversão de peso : lbs -> kg
  '''
  peso_valor = peso_lbs.split( 'lbs', 1 )
  return float( peso_valor[0] ) * 0.453592

In [ ]:
df_fifa['new_weight'] = df_fifa['weight'].astype('str').map( func_converte_peso )
df_fifa.head()

In [ ]:
df_fifa['new_weight'].dtype

In [ ]:
df_fifa['new_weight'].fillna( df_fifa['new_weight'].median(), inplace=True )
df_fifa.head()

In [ ]:
df_fifa['new_weight'].isnull().sum()

In [ ]:
# elimina as colunas originais
df_fifa.drop(inplace=True, columns=['height','weight'])
df_fifa.head()

### demais colunas categóricas

In [ ]:
# nova avaliação da representatividade dos missing values
ds_lin, ds_col = df_fifa.shape
for col in df_fifa.columns:
  soma_mv = df_fifa[col].isnull().sum()
  if soma_mv > 0:
    print(col, ': tipo ', df_fifa[col].dtype, ': qt ', soma_mv, ' : ', (soma_mv/ds_lin)*100, ' %' )

In [ ]:
# list comprehension para mostrar as colunas com missing values e as quantidades correspondentes
l_missing_values_col = [ col for col in df_fifa.columns if df_fifa[col].isnull().sum() > 0 ]
l_missing_values_col

In [ ]:
# 'club' : atribuir texto 'sem clube'
df_fifa['club'].fillna( 'sem clube', inplace=True )
df_fifa['club'].isnull().sum()

In [ ]:
# 'preferred foot' : atribuir o mais freqüente (poderia distribuir aleatoriamente entre 'Right' e 'Left' ?)
df_fifa['preferred foot'].value_counts().plot(kind='bar')

In [ ]:
df_fifa['preferred foot'].fillna( 'Right', inplace=True )
df_fifa['preferred foot'].isnull().sum()

In [ ]:
# 'work rate' : atribuir o mais freqüente : 'Medium/ Medium'
df_fifa['work rate'].value_counts()

In [ ]:
df_fifa['work rate'].value_counts().plot(kind='bar')

In [ ]:
df_fifa['work rate'].fillna( 'Medium/ Medium', inplace=True )
df_fifa['work rate'].isnull().sum()

In [ ]:
# 'body type' - atribuir tipo 'Normal'
df_fifa['body type'].value_counts().plot(kind='bar')

In [ ]:
df_fifa['body type'].fillna( 'Normal', inplace=True )
df_fifa['body type'].isnull().sum()

In [ ]:
# 'position' - atribuir o mais freqüente : 'ST' striker
df_fifa['position'].value_counts().plot(kind='bar')

In [ ]:
df_fifa['position'].fillna( 'ST', inplace=True )
df_fifa['position'].isnull().sum()

In [ ]:
# 'joined' - string - considerar 'nd' não disponível
df_fifa['joined'].unique()

In [ ]:
df_fifa['joined'].fillna( 'nd', inplace=True )
df_fifa['joined'].isnull().sum()

In [ ]:
# 'loaned from' - string - considerar 'nd' não disponível
df_fifa['loaned from'].unique()

In [ ]:
df_fifa['loaned from'].fillna( 'nd', inplace=True )
df_fifa['loaned from'].isnull().sum()

In [ ]:
# 'contract valid until'
df_fifa['contract valid until'].unique()

In [ ]:
df_fifa['contract valid until'].fillna( 'nd', inplace=True )
df_fifa['contract valid until'].isnull().sum()

## 7) Qual a distribuição do número de jogadores por países? Apresente uma tabela com a distribuição.

In [ ]:
df_jogador_pais = df_fifa.groupby('nationality').agg({'id':'count'})
df_jogador_pais.columns = ['qt_jogadores']
df_jogador_pais.sort_values(by='qt_jogadores',ascending=False)

In [ ]:
# tentar pd.crosstab()

## 8) Qual a média de idade dos jogadores por países (variável/atributo 'Nacionality');

In [ ]:
df_idade_pais = df_fifa.groupby('nationality').agg({'id':'count','age':'mean'})
df_idade_pais.columns = ['qt_jogadores','idade']
df_idade_pais.sort_values(by='idade',ascending=False)

## 9) Qual a número de jogadores por idade?

In [ ]:
df_jogador_idade = df_fifa.groupby('age').agg({'age':'count'})
df_jogador_idade.columns = ['qt_idade']
df_jogador_idade.sort_values(by='qt_idade',ascending=False)

In [ ]:
df_fifa['age'].value_counts().plot(kind='bar')

## 10) Quantos jogadores possuem cada clube?

In [ ]:
df_fifa.groupby('club').agg({'id':'count'}).sort_values(by='id',ascending=False)

## 11) Qual a média de idade por clube?

In [ ]:
df_fifa.groupby('club').agg({'id':'count','age':'mean'}).sort_values(by='age',ascending=False)

#df_fifa.groupby('club').agg({'age':['mean','min','max']}).sort_values(by='age',ascending=False)

## 12) Qual a média de salário por país?

coluna 'new_wage'


In [ ]:
df_fifa.groupby('nationality').agg({'nationality':'count', 'new_wage':'mean'}).sort_values(by='new_wage',ascending=False)

## 13) Qual a média de salário por clube?

In [ ]:
df_fifa.groupby('club').agg({'id':'count','new_wage':'mean'}).sort_values(by='new_wage',ascending=False)

## 14) Qual a média de salário por idade?

In [ ]:
df_fifa.groupby('age').agg({'age':'count', 'new_wage':'mean'}).sort_values(by='new_wage',ascending=False)

## 15) Quanto cada clube gasta com pagamento de salários?

In [ ]:
df_fifa.groupby('club').agg({'id':'count','new_wage':'sum'}).sort_values(by='new_wage',ascending=False)

* ---------------------------------------------------------- ponto de início

## 16) Quais são os insight (o que você consegue descobrir) em relação à variável 'Potential' (mede o potencial dos jogadores)?

In [ ]:
df_fifa.columns

In [ ]:
# lista das variáveis não numéricas
for col in df_fifa.columns:
  if ( df_fifa[col].dtype != 'int64' ) and ( df_fifa[col].dtype != 'float64' ):
    print(col)

In [ ]:
# nome das variáveis não numéricas
colunas_categoricas = ['seq', 'id','name','nationality','club','preferred foot','work rate','body type','position','joined','loaned from','contract valid until','new_height','new_weight']

In [ ]:
# cópia do dataframe para estudo de correlação
df_X = df_fifa.copy()

In [ ]:
df_X.drop(columns=colunas_categoricas, inplace=True)
df_X.head()

In [ ]:
df_X.columns

In [ ]:
# reordena colunas
lista_colunas = [ 'acceleration', 'age', 'agility', 'aggression', 'balance', 'ballcontrol', 'composure', 'crossing',
 'curve', 'dribbling', 'finishing', 'fkaccuracy', 'gkdiving', 'gkhandling', 'gkkicking', 'gkpositioning',
 'gkreflexes', 'headingaccuracy', 'interceptions', 'international reputation', 'jumping', 'longpassing',
 'longshots', 'marking', 'new_wage', 'new_value', 'new_release_value', 'overall', 'penalties', 'positioning',
 'potential', 'reactions', 'shortpassing', 'shotpower', 'skill moves', 'slidingtackle', 'special', 'sprintspeed',
 'stamina', 'standingtackle', 'strength', 'vision', 'volleys', 'weak foot' ]

df_X = df_X[ lista_colunas ]

In [ ]:
df_X.info()

In [ ]:
# geração de dados reescalonados : coluna_rescala = ( val - media ) / desvio
for col in df_X.columns:
  media = df_X[col].mean()
  desvio = df_X[col].std()
  df_X[col] = df_X[col].sub( media )
  df_X[col] = df_X[col].div( desvio )

df_X.head()

In [ ]:
# df_X.corr() : Compute pairwise correlation of columns, excluding NA/null values.
# np.triu() : Return a copy of a matrix with the elements below the k-th diagonal zeroed.
# np.ones() : Return a new array of given shape and type, filled with ones.

# matriz de correlação entre colunas numéricas - Pearson
matriz_cor = df_X.corr().where( np.triu( np.ones( df_X.corr().shape ), k=1 ).astype( np.bool ) )
matriz_cor

In [ ]:
# conjunto de colunas correlacionadas
set_col_cor = set()

for i in range( len( matriz_cor.columns ) ):
  for j in range( i ):
    if abs( matriz_cor.iloc[i,j] ) > 0.8:
      nome_col = matriz_cor.columns[i]
      set_col_cor.add( nome_col )

set_col_cor

In [ ]:
# diminui tamanho da fonte de caracteres
sns.set(font_scale=0.8)

# Create a figure and a set of subplots.
fig, ax = plt.subplots( figsize = ( 25, 25 ) )

# numpy.zeros_like(a, dtype=None, order='K', subok=True, shape=None) : Return an array of zeros with the same shape and type as a given array.
mask = np.zeros_like( df_X.corr().abs() )

# numpy.triu_indices_from(arr, k=0)[source] : Return the indices for the upper-triangle of arr.
mask[ np.triu_indices_from( mask ) ] = 1

# seaborn.heatmap(data, *, vmin=None, vmax=None, cmap=None, center=None, robust=False, annot=None, fmt='.2g',
# annot_kws=None, linewidths=0, linecolor='white', cbar=True, cbar_kws=None, cbar_ax=None, square=False,
# xticklabels='auto', yticklabels='auto', mask=None, ax=None, **kwargs)
#
# Plot rectangular data as a color-encoded matrix.
sns.heatmap( df_X.corr().abs(), mask = mask, ax = ax, cmap = 'coolwarm', annot = True, fmt = '.2f', center = 0 )

In [ ]:
# scatter plot sem regressão
#sns.pairplot( df_X, kind = 'scatter' )
#plt.show()

## 17) Quais os insights em relação à variável overall (nota média do atleta) por idade, clube e país?

In [ ]:
df_fifa.groupby(['club']).agg({'overall':'mean'}).sort_values(by=['overall'], ascending=False)

In [ ]:
df_fifa.groupby(['nationality']).agg({'id':'count','overall':'mean'}).sort_values(by=['overall'], ascending=False)

In [ ]:
df_fifa.groupby(['age']).agg({'id':'count','overall':'mean'}).sort_values(by=['overall'], ascending=False)

In [ ]:
df_fifa.groupby(['overall']).agg({'nationality':'count','club':'count'}).sort_values(by=['overall'], ascending=False)

In [ ]:
# no heatmap, correlação age x overall = 0.45
sns.jointplot( df_X['age'], df_X['overall'], kind='kde' )
plt.show()

* ---------------------------------------------------------- ponto final

## 18) Quais são os melhores clubes se levarmos em consideração as variáveis Potential e Overall?

In [ ]:
df_fifa.groupby('club').agg({'potential':'sum','overall':'sum'}).sort_values(by=['potential','overall'], ascending=False)

## 19) Apresente o ranking dos goleiros (use a variável/atributo 'Preferred Positions'. por Potencial, Overall. Estamos à procura de 'GK'.

In [ ]:
df_fifa.columns

In [ ]:
df_fifa['position'].value_counts()

In [ ]:
df_fifa[ df_fifa['position'] == 'GK' ].groupby('name').agg({'potential':'max','overall':'max'}).sort_values(by=['potential','overall'], ascending=False)

## 20) Quem são os jogadores mais rápidos (variável/atributo 'Sprint speed'?

In [ ]:
df_fifa['sprintspeed'].median()

In [ ]:
df_fifa.groupby('name').agg({'sprintspeed':'max'}).sort_values(by=['sprintspeed'], ascending=False).head(10)

## 21) Quem são os 5 melhores jogadores em termos de chute (força para chutar) (use a variável/atributo 'Shot power')?

In [ ]:
df_fifa['shotpower'].value_counts()

In [ ]:
df_fifa.groupby('name').agg({'shotpower':'max'}).sort_values(by=['shotpower'], ascending=False).head(5)

In [ ]:
# se fosse por salário
df_fifa['new_wage'].value_counts()

In [ ]:
df_fifa.groupby('name').agg({'new_wage':'max'}).sort_values(by=['new_wage'], ascending=False).head(5)

## 22) Quem são os outliers em termos de salário?

In [ ]:
df_fifa['new_wage'].describe()

In [ ]:
wage_percent = np.percentile( df_fifa['new_wage'], q=[1,25,50,75,99] )
wage_percent

In [ ]:
wage_q1 = wage_percent[1]
wage_q3 = wage_percent[3]
print('q1 ',wage_q1,' q3 ',wage_q3)

In [ ]:
wage_linf = wage_q1 - (1.5 * (wage_q3 - wage_q1))
wage_lsup = wage_q3 + (1.5 * (wage_q3 - wage_q1))
print('wage inf ',wage_linf,' wage sup ',wage_lsup)

In [ ]:
# outliers sup
df_fifa.loc[ df_fifa['new_wage'] > wage_lsup ][['name','new_wage']]

In [ ]:
# outliers inf
df_fifa.loc[ df_fifa['new_wage'] < wage_linf ]['name']

## 23) Quem são os outliers em termos de potência no chute?

In [ ]:
df_fifa['shotpower'].describe()

In [ ]:
sp_percent = np.percentile( df_fifa['shotpower'], q=[1,25,50,75,99] )
sp_percent

In [ ]:
sp_q1 = sp_percent[1]
sp_q3 = sp_percent[3]
print('q1 ',sp_q1,' q3 ',sp_q3)

In [ ]:
sp_linf = sp_q1 - (1.5 * (sp_q3 - sp_q1))
sp_lsup = sp_q3 + (1.5 * (sp_q3 - sp_q1))
print('sp inf ',sp_linf,' sp sup ',sp_lsup)

In [ ]:
# outliers sup
df_fifa.loc[ df_fifa['shotpower'] > sp_lsup ]['name']

In [ ]:
# outliers inf
df_fifa.loc[ df_fifa['shotpower'] < sp_linf ][['name','shotpower']]

* Exercício 2 do notebook de visualização


Usando o dataframe FIFA, responda:

(1) Mostre o gráfico de barras com o número de jogadores por clube;

(2) Mostre o boxplot/histograma dos salários dos atletas para os clubes Real Madrid, Barcelona Paris Saint-Germain Bayern Munich e Juventus.

In [ ]:
# (1) Mostre o gráfico de barras com o número de jogadores por clube

df_fifa.groupby('club').agg({'id':'count'}).sort_values(by='id',ascending=False).head(10).plot(kind='barh')


In [ ]:
# histograma de salário por clube
# (2) Mostre o boxplot/histograma dos salários dos atletas para os clubes Real Madrid, Barcelona Paris Saint-Germain Bayern Munich e Juventus.
df_fifa.loc[ df_fifa['club'] == 'Real Madrid' ][ 'new_wage' ].plot( kind='hist', bins=100, stacked=True)

In [ ]:
df_fifa.loc[ df_fifa['club'] == 'FC Barcelona' ][ 'new_wage' ].plot( kind='hist', bins=100, stacked=True)

In [ ]:
df_fifa.loc[ df_fifa['club'] == 'Paris Saint-Germain' ][ 'new_wage' ].plot( kind='hist', bins=100, stacked=True)

In [ ]:
df_fifa.loc[ df_fifa['club'].str.contains('Bayern') ][ 'new_wage' ].plot( kind='hist', bins=100, stacked=True)

In [ ]:
df_fifa.loc[ df_fifa['club'] == 'Juventus' ][ 'new_wage' ].plot( kind='hist', bins=100, stacked=True)

In [ ]:
# df_fifa[ df_fifa['club'].isin[ 'FC Barcelona','Paris Saint-Germain','FC Bayern Munchen','Real Madrid','Juventus' ] ]#.plot( kind='hist', bins=100, stacked=True)